# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.3, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

In [3]:
%pip install -q findspark pyspark

Note: you may need to restart the kernel to use updated packages.


# Start a Local Cluster

In [4]:
import findspark
from pyspark.sql import SparkSession
spark_url = 'local'
spark = SparkSession.builder.master(spark_url).appName("MyApp").getOrCreate()

# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

In [5]:
path = 'netflix-rotten-tomatoes-metacritic-imdb.csv'
df = spark.read.option("delemiter",";").option("header",True).csv(path)
df.show(3)

+-------------------+--------------------+--------------------+----------------+---------------+----------------+--------------------+------------+---------------+--------------------+--------------------+-----------+----------+---------------------+----------------+---------------+--------------------+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+
|              Title|               Genre|                Tags|       Languages|Series or Movie|Hidden Gem Score|Country Availability|     Runtime|       Director|              Writer|              Actors|View Rating|IMDb Score|Rotten Tomatoes Score|Metacritic Score|Awards Received|Awards Nominated For| Boxoffice|Release Date|Netflix Release Date|    Production House|        Netflix Link|           IMDb Link|             Summary|IMDb Votes|               Image|              

## What is the maximum and average of the overall hidden gem score?

In [8]:
max_hidden_gem_score = df.select('Hidden Gem Score').agg({"Hidden Gem Score":'max'}).collect()[0][0]
# df.select('Hidden Gem Score').agg({"Hidden Gem Score":'max'}).collect()
avg_hidden_gem_score = df.select('Hidden Gem Score').agg({"Hidden Gem Score":'avg'}).collect()[0][0]
print(max_hidden_gem_score,avg_hidden_gem_score)

[Row(max(Hidden Gem Score)='9.8')]

## How many movies that are available in Korea?

In [16]:
# df[df['Languages'].contains('Korean')].count()
print(df.filter(df['Languages'].contains('Korean')).count())

735


## Which director has the highest average hidden gem score?

In [9]:
print(df.groupBy('Director').agg({'Hidden Gem Score':'avg'}).orderBy('avg(Hidden Gem Score)',ascending=False).first()['Director'])


Dorin Marcu


## How many genres are there in the dataset?

In [18]:
from pyspark.sql.functions import split,explode

genre_df = df.withColumn('Genre',split(df['Genre'],', ')) #replace column
# genre_df.show(3)

# genre = genre_df.select(explode(genre_df['Genre'])).distinct()
# genre.show(3)

# print(genre.count())

AnalysisException: [DATATYPE_MISMATCH.FILTER_NOT_BOOLEAN] Cannot resolve "split(Genre, , , -1)" due to data type mismatch: Filter expression "split(Genre, , , -1)" of type "ARRAY<STRING>" is not a boolean.;
Filter split(Genre#18, , , -1)
+- Relation [Title#17,Genre#18,Tags#19,Languages#20,Series or Movie#21,Hidden Gem Score#22,Country Availability#23,Runtime#24,Director#25,Writer#26,Actors#27,View Rating#28,IMDb Score#29,Rotten Tomatoes Score#30,Metacritic Score#31,Awards Received#32,Awards Nominated For#33,Boxoffice#34,Release Date#35,Netflix Release Date#36,Production House#37,Netflix Link#38,IMDb Link#39,Summary#40,... 5 more fields] csv
